# 캐치 크롤링
- 크롤링할 컬럼: 기업명, 기업형태, 매출액, 사원수, 업종분류, 설립연도
- 절차
1. url로 기업명 입력
2. 기업정보 사이트 접속
3. 내용 크롤링
4. concat
5. 다음 url 입력

위 절차를 반복합니다.

In [ ]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
df = pd.read_csv('./programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.
df_corp_name = df['기업명']  # 기업명 불러오기

In [ ]:
# 임시 프레임
catch_res = pd.DataFrame(columns=['기업명','기업형태','매출액','사원수','업종분류','설립연도'])

In [ ]:
# 크롤링 프로그램
for name in df_corp_name:
    try:
        url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'

        driver= webdriver.Chrome()
        driver.get(url)

        driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()
        time.sleep(0.2)

        # 기업명
        corp_name = name
        # 기업형태
        corp_type = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[2]/td[1]').text
        # 매출액
        sales_money = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[1]').text
        # 사원수
        employee = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[2]').text
        # 업종분류
        try:
            sector = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[5]/td[1]').text
        except:
            sector = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[4]/td[1]').text
        # 설립연도
        corp_year = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[1]/td[2]').text

        catch_temp = pd.DataFrame([corp_name, corp_type, sales_money, employee, sector, corp_year],
                                   index=['기업명','기업형태','매출액','사원수','업종분류','설립연도']).T
        catch_res = pd.concat([catch_res, catch_temp])

        time.sleep(0.2)
    except:
        pass
    
    driver.quit()

In [ ]:
catch_res.to_csv('./catch_result2.csv')  # 파일명만 변경하여 저장하시면 됩니다.

# 코드 수정

In [ ]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
df = pd.read_csv('./programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.
df_corp_name = df['기업명']  # 기업명 불러오기

In [ ]:
# 임시 프레임
catch_res = pd.DataFrame(columns=['기업명','기업형태','매출액','사원수','업종분류','설립연도'])

In [ ]:
# 크롤링 프로그램
for name in df_corp_name:
    url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'  # 기업명으로 URL불러오기

    driver= webdriver.Chrome()
    driver.get(url)
    
    try:
        driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()
        time.sleep(0.2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기업명
        corp_name = name
        corp_year = soup.select('td')[1].text  # 설립연도
        corp_type = soup.select('td')[2].text  # 기업 형태
        if len(soup.select('td')) == 10:
            sales_money = soup.select('td')[4].text  # 매출액
            employee = soup.select('td')[8].text  # 사원수
            sector = soup.select('td')[9].text  # 업종분류
        elif len(soup.select('td')) == 8:
            sales_money = soup.select('td')[4].text  # 매출액
            employee = soup.select('td')[5].text  # 사원수
            sector = soup.select('td')[6].text  # 업종분류        
        else:
            employee = soup.select('td')[4].text  # 사원수
            sector = soup.select('td')[5].text  # 업종분류

        catch_temp = pd.DataFrame([corp_name, corp_type, sales_money, employee, sector, corp_year],
                                   index=['기업명','기업형태','매출액','사원수','업종분류','설립연도']).T
        catch_res = pd.concat([catch_res, catch_temp])

        time.sleep(0.2)
    except:
        pass
    
driver.quit()

In [ ]:
catch_res.to_csv('./catch_result.csv')  # 파일명만 변경하여 저장하시면 됩니다.

# 2차 수정(최종본)
※ 주의사항: 사이트가 불안정하여 오류가 날 경우 time.sleep을 넉넉하게 설정하여 주세요

In [1]:
import pandas as pd
import numpy as np
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [6]:
corp_df = pd.read_csv('./programmers.csv', index_col=0)  # 파일명만 변경하여 그대로 돌리시면 됩니다.
corp_df.reset_index(drop=True, inplace=True)  # 인덱스 재설정
# corp_list = corp_df['기업명'].drop_duplicates()  # 기업명 중복제거

In [7]:
corp_df.columns

Index(['사원수', '주소', '직무', '이용하는 기술스택', '자격요건', '직급', '우대사항', '해당 페이지 URL',
       '기업 홈페이지 URL'],
      dtype='object')

In [ ]:
# 결과를 저장할 빈 프레임
df_res = pd.DataFrame(columns=['기업규모', '기업형태', '당기손익', '대표자', '매출액','사원수', '상세업종', '설립일', '신용등급', '영업이익', '카테고리'])  

In [ ]:
# 크롤링 프로그램
for name in corp_list:
    url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'  # 기업명으로 URL불러오기

    driver= webdriver.Chrome()
    driver.get(url)
    try:
        driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()
        time.sleep(0.3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        list = ['기업명']  # 컬럼이 될 리스트
        list2 = [name]  # 값이 될 리스트
        for i in soup.find_all('tbody')[0].select('th'):
            list.append(i.text)

        for j in soup.find_all('tbody')[0].select('td'):
            list2.append(j.text)
    except:
        pass
    df_temp = pd.DataFrame([list2], columns=list)
    df_temp.reset_index(drop=True, inplace=True)
    df_res = pd.concat([df_res, df_temp], ignore_index=True)  # 결과 DataFrame과 새로운 DataFrame을 결합
    df_res.reset_index(drop=True, inplace=True)

driver.quit()

# 캐치 전처리

In [ ]:
df_result = df_res[['기업명','기업규모','매출액','사원수','설립일','상세업종']]  # 필요한 컬럼만 추출
df_result.rename(columns={'설립일': '설립연도', '상세업종': '업종분류', '기업규모': '기업형태'}, inplace=True)  # 컬럼이름 변경

In [ ]:
# 매출액 전처리
m_list = []  
m_list2 = []  
m_list3 = []
# '조'와 '억'만 남기기
for m in df_result['매출액']:
    if m == None:  # 결측치인 경우 str타입으로 변경
        m_list.append(str(m))
    elif '재무제표' in str(m):
        m_list.append(re.sub('\n', '', m.split(':')[-1].strip()))
    else:
        m_list.append(re.sub('\n', '', str(m)).split()[0])
# 억에서 , 제외하기
for m2 in m_list:
    m_list2.append(re.sub(',', '', m2))

# 조, 억, 만원을 만의단위로 변경하기
for m3 in m_list2:
    if m3 == '-' or m3 == 'nan':  # 결측치 처리
        m_list3.append(None)
    else:
        조_matches = re.findall(r'(\d+)조', m3)
        조 = int(조_matches[0]) * 10**8 if 조_matches else 0

        억_matches = re.findall(r'(\d+)억', m3)
        억 = int(억_matches[0]) * 10**4 if 억_matches else 0

        만원_matches = re.findall(r'(\d+)만원', m3)
        만원 = int(만원_matches[0]) if 만원_matches else 0

        m_list3.append(조 + 억 + 만원)
        
df_result['매출액'] = m_list3

In [ ]:
# 사원수
peo_list =[]

for p in df_result['사원수']:
    try:
        if '-명' in p:
            peo_list.append(None)
        elif '명' in p:
            peo_list.append(int(re.findall(r'\d+', re.sub(',', '', p))[0]))
    except:
        peo_list.append(None)

df_result['사원수'] = peo_list

In [ ]:
# 설립연도
year_list = []

for y in df_result['설립연도']:
    try:
        year_list.append(int(re.findall(r'\d+', y)[0]))
    except:
        year_list.append(None)
        
df_result['설립연도'] = year_list

In [ ]:
# 인덱스 재설정
df_result.reset_index(drop=True, inplace=True)